In [1]:
from SIV_library.lib import block_match, moving_window_array, correlation_to_displacement, get_field_shape, get_x_y, SIVDataset
from torchPIV.PIVbackend import OfflinePIV
from SIV_library.optical_flow_2021 import HornSchunck

import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt

os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
sys.path.append("../../")

%load_ext autoreload
%autoreload 2

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

device = "cpu"
if device == "cuda":
    torch.cuda.empty_cache()

dataset = SIVDataset("../Test Data/Cilinder_PROCESSED", None)  # (300, 2500, 400, 1600)

cuda


In [28]:
def vector_field(lib_res, corr_res, sad_res, opt_res) -> None:
    fig, (lib, corr, sad, opt) = plt.subplots(1, 4)
 
    frame = dataset[0][0].cpu().numpy()
    lib.imshow(frame, cmap='gray')
    corr.imshow(frame, cmap='gray')
    sad.imshow(frame, cmap='gray')
    opt.imshow(frame, cmap='gray')
    
    lib_res[1] = np.flip(lib_res[1], axis=0)
    corr_res[3] *= -1
    sad_res[3] *= -1
    
    lib.quiver(*lib_res, color='red', scale=10, scale_units='xy')
    corr.quiver(*corr_res, color='red', scale=1, scale_units='xy')
    sad.quiver(*sad_res, color='red', scale=0.3, scale_units='xy')
    opt.quiver(*opt_res, color='red', scale=0.03, scale_units='xy')
    
    lib.set_title('TorchPIV')
    corr.set_title('Correlation')
    sad.set_title('SAD')
    opt.set_title('Horn-Schunck')
    
    lib.set_axis_off()
    corr.set_axis_off()
    sad.set_axis_off()
    opt.set_axis_off()
    
    fig.tight_layout()
    plt.show()

In [51]:
def all_frames(window_size, overlap):
    piv_gen = OfflinePIV(
                folder="../Test Data/Cilinder_PROCESSED",
                device=torch.cuda.get_device_name(),
                file_fmt="jpg",
                wind_size=window_size,
                overlap=overlap,
                dt=12,
                scale = 1.,
                multipass=2,
                multipass_mode="DWS",
                multipass_scale=2.,
                folder_mode="sequential"
            )
    piv_gen._dataset = dataset
    
    hs = HornSchunck([dataset[0]], 100., 10, device)
    
    for idx, data in enumerate(dataset):
        if idx == 0:
            image_a, image_b = data
            image_a, image_b = image_a.to(device), image_b.to(device)
            
            n_rows, n_cols = get_field_shape(image_a.shape, window_size, overlap)
            window = moving_window_array(image_a, window_size, overlap)
            
            area = moving_window_array(image_b, window_size, overlap, area=(30, 30, 30, 30))
            
            x, y= get_x_y(image_a.shape, window_size, overlap)
            x, y = x.reshape(n_rows, n_cols), y.reshape(n_rows, n_cols)
            
            hs.optical_flow()
            u_o, v_o = hs.velocities[0, 0][y, x], hs.velocities[0, 1][y, x]
            
            lib = next(piv_gen())
            
            corr = block_match(window, area, 0)
            u_c, v_c = correlation_to_displacement(corr, n_rows, n_cols, 0)
            
            sad = block_match(window, area, 1)
            u_s, v_s = correlation_to_displacement(sad, n_rows, n_cols, 1)
            
            lib_res, corr_res, sad_res, opt_res = np.array(lib), np.array((x, y, u_c, v_c)), np.array((x, y, u_s, v_s)), np.array((x, y, u_o, v_o))
    return lib_res, corr_res, sad_res, opt_res

In [52]:
lib_res, corr_res, sad_res, opt_res = all_frames(64, 32)

100%|██████████| 1/1 [00:00<00:00,  1.20it/s]


Load time 0.036 sec Iteration finished in 0.066 sec 

SAD: 100%|██████████| 61/61 [01:09<00:00,  1.13s/it]


KeyboardInterrupt: 

In [43]:
vector_field(lib_res, corr_res, sad_res, opt_res)